## Soportes y Resistencias automaticas de BTC



In [29]:
import requests
import pandas as pd
from datetime import datetime

# Genera el timestamp actual en milisegundos para endTime
end_time = int(datetime.now().timestamp() * 1000)

# Parámetros de la solicitud
url = "https://api.binance.com/api/v3/klines"
params = {
    "symbol": "BTCUSDT",       # Par de trading
    "interval": "1w",          # Intervalo de velas: '1m', '5m', '1h', '1d', etc.
    "startTime": 1577836800000, # Tiempo de inicio en formato timestamp (ms)
    "endTime": end_time,   # Tiempo de fin en formato timestamp (ms)
    "limit": 300         # Número máximo de registros a devolver
}

# Realizar la solicitud GET a la API
response = requests.get(url, params=params)
data = response.json()

# Seleccionar solo las primeras 5 columnas (timestamp, open, high, low, close)
filtered_data = [[item[0], item[1], item[2], item[3], item[4]] for item in data]

# Crear el DataFrame de pandas con las columnas necesarias
df = pd.DataFrame(filtered_data, columns=['timestamp', 'open', 'high', 'low', 'close'])

# Convertir la columna 'timestamp' a un formato de fecha legible
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Convertir las columnas numéricas a tipo float
df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)

# Mostrar los primeros registros del DataFrame
print(df.tail(10))


     timestamp      open      high       low     close
244 2024-09-09  54869.95  60625.00  54591.96  59132.00
245 2024-09-16  59132.00  64133.32  57493.30  63578.76
246 2024-09-23  63578.76  66498.00  62538.75  65602.01
247 2024-09-30  65602.01  65618.80  59828.11  62819.91
248 2024-10-07  62819.91  64478.19  58946.00  62870.02
249 2024-10-14  62870.02  69400.00  62457.81  69031.99
250 2024-10-21  69032.00  69519.52  65260.00  68021.70
251 2024-10-28  68021.69  73620.12  67478.73  68775.99
252 2024-11-04  68775.99  81500.00  66835.00  80370.01
253 2024-11-11  80370.01  89940.00  80216.01  87002.02


In [30]:
# Definir una ventana para buscar máximos y mínimos locales
window = 10  # Ajusta el tamaño de la ventana según el nivel de detalle que desees

# Listas para almacenar los niveles de soporte y resistencia
resistencias = []
soportes = []

# Recorrer los datos de precios
for i in range(window, len(df) - window):
    max_local = max(df['high'][i - window:i + window + 1])
    min_local = min(df['low'][i - window:i + window + 1])

    # Si el precio actual es un máximo local, se marca como resistencia
    if df['high'][i] == max_local:
        resistencias.append((df['timestamp'][i], df['high'][i]))
    
    # Si el precio actual es un mínimo local, se marca como soporte
    if df['low'][i] == min_local:
        soportes.append((df['timestamp'][i], df['low'][i]))

# Crear DataFrames para soportes y resistencias
df_resistencias = pd.DataFrame(resistencias, columns=['timestamp', 'resistencia'])
df_soportes = pd.DataFrame(soportes, columns=['timestamp', 'soporte'])

# Mostrar los primeros resultados
print("Resistencias detectadas:")
print(df_resistencias.head())

print("\nSoportes detectados:")
print(df_soportes.head())


Resistencias detectadas:
   timestamp  resistencia
0 2021-04-12     64854.00
1 2021-11-08     69000.00
2 2022-03-28     48189.84
3 2023-07-10     31804.20
4 2024-03-11     73777.00

Soportes detectados:
   timestamp   soporte
0 2021-06-21  28805.00
1 2022-01-24  32917.17
2 2022-06-13  17622.00
3 2022-11-21  15476.00
4 2023-06-12  24800.00


In [31]:
import plotly.graph_objects as go

# Crear una figura para el gráfico de velas
fig = go.Figure(data=[go.Candlestick(
    x=df['timestamp'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close']
)])

# Agregar líneas horizontales para cada resistencia
for resistencia in df_resistencias['resistencia']:
    fig.add_shape(
        type="line",
        x0=df['timestamp'].min(), y0=resistencia,
        x1=df['timestamp'].max(), y1=resistencia,
        line=dict(color="red", width=1.5, dash="dot"),
        name="Resistencia"
    )

# Agregar líneas horizontales para cada soporte
for soporte in df_soportes['soporte']:
    fig.add_shape(
        type="line",
        x0=df['timestamp'].min(), y0=soporte,
        x1=df['timestamp'].max(), y1=soporte,
        line=dict(color="green", width=1.5, dash="dot"),
        name="Soporte"
    )

# Configurar el diseño del gráfico con un mayor tamaño
fig.update_layout(
    title="Gráfico de velas BTC/USDT",
    xaxis_title="Fecha",
    yaxis_title="Precio (USD)",
    xaxis_rangeslider_visible=False,  # Oculta el deslizador de rango
    width=1500,  # Ancho del gráfico
    height=800   # Altura del gráfico
)

# Mostrar el gráfico
fig.show()
